# Backend for Peaking Analysis

In [43]:
#Import data science libraries
import pandas as pd
import numpy as np
import simplejson
#Calculate current year for use in peaking criteria 
from datetime import date
current_year = date.today().year
#Import modules for sending emails
import smtplib
from email.message import EmailMessage
#Set display options
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows',1000)

In [44]:
def read_in_data_from_master_emissions_tracker(path):
    """
    Reads in data from 2017 peaking analysis and reshapes for use in programme.
    INPUT: Excel file (.xlsx)
    OUTPUT: Pandas DataFrame
    """
    #df = pd.read_excel(path, sheet_name='Master data_GHG & targets')
    df = pd.read_excel(path, sheet_name='All raw GHG data_2018',header = 1)
    df = df[['City name tidy up','Source_Protocol','Inventory\n_year_2', 'Emissions\n_mtCO2e','Use in dashboard 2','Updated by city in 2018']]
    df.rename(columns={'City name tidy up':'City', 'Source_Protocol':'Protocol','Inventory\n_year_2':'Year','Emissions\n_mtCO2e':'Emissions','Updated by city in 2018':'Verified by city'},inplace = True)
    df = df[df['Use in dashboard 2']== 'Yes'] #Filter out invalid rows
    
    def map_protocols_to_data_sources(x):
        if x == 'C40_GPC':
            return 1
        if x == 'City_GPC':
            return 2
        if x == 'CDP_GPC':
            return 3
        if x == 'Target_Other':
            return 5
        if x == 'CDP_Other':
            return 6
        if x == 'City_Other':
            return 7 
    
    df['Data Source'] = df['Protocol'].apply(map_protocols_to_data_sources)
    df['Verified by city'].fillna('No',inplace=True) #Fill data gaps with "No"
    #df = df[df['Data Source']!= 1] #Remove all C40 GPC records
    df = df[df['Year']!= 0] #Remove all records without year data
    df['Emissions'].fillna(0, inplace = True)
    df = df[df['Emissions']!=0] #Remove all records without emissions data
    df = df[df['City']!='Basel'] #Remove records where city is Basel
    df = df[['City','Data Source','Protocol','Verified by city','Year','Emissions']].sort_values(['City','Year']) #Reorder columns and sort
    return df

In [47]:
def combine_gpc_and_non_gpc_data_sources(df):
    """
    Calculates All GPC Considered and All non-GPC considered rows by combining data sources for each city. Function 
    constructs All GPC Considered rows by backfilling data gaps in C40 GPC data sources with CDP GPC and City GPC data.
    Function constructuct All Non GPC Considered rows by backfilling data gaps in C40 Target Baseline Other data sources
    with CDP Other and City Other. 
    INPUT: Master DataFrame with GPC Tracker and Master GHG data
    OUTPUT: DataFrame with additional All GPC Considered and All non-GPC considered rows
    """
        
    def reshape_data(df):
        """
        Reshapes DataFrame so that each city and datasource has a unique row with years as a column header
        INPUT: DataFrame
        OUTPUT: Reshaped DataFrame
        """
        df = df.pivot_table(values='Emissions', index=['City','Data Source','Protocol','Verified by city'], columns='Year', aggfunc='first').reset_index()#.fillna(0) REPLACE!
        df.columns.name = None #set column index name to none 
        return df
    
    def calculate_gpc_and_non_gpc_combinations(df):
        #Create 2 dataframes for GPC and non-GPC data 
        df1 = df.copy()
        df2 = df.copy()
        df1 = df[df['Data Source'] <= 3]
        df2 = df[df['Data Source'] >= 4]
        #Create variable for columns with emissions data 
        cols = df.columns.difference(['City','Data Source','Protocol','Verified by city'])

        def combine_rows_for_each_city(df, cols, rows):
            #Select rows where there is at least 1 city
            df = df.groupby("City").filter(lambda x: len(x) > 1) 
            
            for col in cols:
                df[col] = df.groupby(['City'], sort=False)[col].apply(lambda x: x.fillna(method='bfill'))
            df = df.drop_duplicates(subset='City', keep="first")

            if rows == 'all_gpc':
                df['Data Source'] = 4
                df['Protocol'] = 'All GPC considered'
            if rows == 'all_non_gpc':
                df['Data Source'] = 8
                df['Protocol'] = 'All non GPC considered'
            return df

        df_all_gpc_considered = combine_rows_for_each_city(df1,cols,'all_gpc')
        df_all_non_gpc_considered = combine_rows_for_each_city(df2,cols,'all_non_gpc')    

        #Combine dataframes into a master dataset
        df = df.append([df_all_gpc_considered, df_all_non_gpc_considered]).reset_index(drop=True)
        df = df.sort_values(['City','Data Source'], ascending = [True,True])
        #df = df.drop_duplicates(subset = cols, keep='first').reset_index(drop = True) 
        df.fillna(0, inplace=True)
        return df
    
    df = reshape_data(df)
    df = calculate_gpc_and_non_gpc_combinations(df)
    df.fillna(0, inplace=True)
    
    return df

In [48]:
def calculate_peak_emissions(df, current_year):
    """
    Analyses city GHG emissions to determine if they have peaked
    INPUT: DataFrame containing peaking analysis and GPC Tracker GHG emissions
    OUTPUT: DataFrame with assessment of whether each city has peaked
    """

    def calculate_peaking_parameters(df):
        """
        Caclulate parameters used to assess whether city has peaked emissions
        INPUT: DataFrame
        OUTPUT: DataFrame with peakign parameters as 4 additional columns
        """
        cols = df.columns.difference(['City','Data Source','Protocol','Verified by city'])
        df['Num data points'] = df[cols].gt(0).sum(axis=1)
        df['Max emissions'] = df[cols].max(axis =1)
        df['Max emissions year'] = df[cols].idxmax(axis =1)
        df['Recent emissions'] = df[cols].apply(lambda x: x.iloc[x.nonzero()].iloc[-1], axis=1)
        df['Recent emissions year'] = df[cols].apply(lambda x: x.iloc[x.nonzero()].index[-1], axis=1)
        return df
    
    def apply_peaking_criteria(df, current_year):
        """
        Calculates peaking criteria using peaking parameters
        INPUT: DataFrame
        OUTPUT: DataFrame with Boolean assessment for each peaking criteria
        """
        #Peaking Criteria 1: At least 3 years of data available?
        df['PC1'] = (df['Num data points'] >= 3)
        #Peaking Criteria 2: Max emissions >5 years before recent inventory?
        df['PC2'] = (df['Recent emissions year'] - df['Max emissions year'] >= 5)
        #Peaking Criteria 3: Recent inventory < 5 years old
        df['PC3'] = (current_year - df['Recent emissions year'] <= 5)
        #Peaking Criteria 4: Max emissions >10% higher than recent inventory
        df['PC4'] = ((df['Max emissions']-df['Recent emissions'])/df['Recent emissions']) >= 0.1
        return df
    
    def calculate_peak_emissions_status(df):
        """
        Analyses peaking criteria to assess whether city has peaked
        INPUT: DataFrame
        OUTPUT: DataFrame with peaking assessment as additional columne
        """
        
        def evaluate_peaking_criteria(x):
            
            if x['PC1'] & x['PC2'] & x['PC3'] & x['PC4']: #If all peaking criteria are TRUE returns 'PEAKED'
                return 'PEAKED'
            elif x['PC1'] & x['PC3'] & (not x['PC4']): #If first 3 peaking critiera are TRUE and PC4 are FALSE return 'NOT PEAKED'
                return 'NOT PEAKED'
            else: #Otherwise returns 'CANNOT TELL'
                return 'UNKNOWN'
            
        #Calculates columns by applying above function across rows 
        df['Peak Status'] = df.apply(evaluate_peaking_criteria, axis = 1)
        return df 
    
    def rename_columns(df):
        """
        Rename columns for ease of understanding
        """
        df.rename(columns ={
            'PC1':'PC1: At least 3 year of data available',
            'PC2':'PC2: Max emissions >5 years before recent inventory',
            'PC3':'PC3: Recent inventory <5 years old',
            'PC4':'PC4: Max emissions <10% higher than recent inventory'},
            inplace = True)
        return df 
  
    df = calculate_peaking_parameters(df)
    df = apply_peaking_criteria(df, current_year)
    df = calculate_peak_emissions_status(df)
    df = rename_columns(df)
    return df

In [49]:
def select_cities_to_use_in_dashboard(df):
    """
    Selects data sources to use in dashboard for each city
    INPUT: DataFrame 
    OUTPUT: DataFrame with sorted cities conaining NO duplicates for use in peaking analysis dashboard 
    """    

    def select_cities(df):
        """
        Selects initial datasources by ordering Fataframe by City, Verfiedy by City and Data Source columns and 
        selecting the first instance of PEAKED, NOT PEAKED or UNKNOWN depending on  conditions which count the 
        number of peak statuses for each city.
        INPUT: DataFrame
        OUTPUT: DataFrame with additional column identifying which record to use for each city 
        """
        
        #Order dataframe by city and data source 
        df['Verified by city'] = pd.Categorical(df['Verified by city'],categories=["Yes","No"],ordered=True)
        df = df.sort_values(['City','Verified by city','Data Source'], ascending = [True, True, True]).reset_index(drop=True) 
        
        #Initialise parameters used in loop 
        ref_city = df['City'][0]
        peak_list, not_peak_list, unknown_peak_list = [],[],[]

        for index, row in df.iterrows(): 
            city = row['City']
            peak_status = row['Peak Status'] 

            if city == ref_city:
                if peak_status == 'PEAKED':
                    peak_list.append(index)
                if peak_status == 'NOT PEAKED':
                    not_peak_list.append(index)
                if peak_status == 'UNKNOWN':
                    unknown_peak_list.append(index)

            if city != ref_city:
                if len(peak_list) > 0 and len(peak_list) >= len(not_peak_list): 
                    selected_index = peak_list[0]
                elif len(not_peak_list) > len(peak_list):
                    selected_index = not_peak_list[0]
                else:
                    selected_index = unknown_peak_list[0]
                    
                df.loc[selected_index,'Use for dashboard?'] = 'Y'

                #Reinitialise values for use in loop 
                ref_city = city
                peak_list, not_peak_list, unknown_peak_list = [],[],[]
                
                if peak_status == 'PEAKED':
                    peak_list.append(index)
                if peak_status == 'NOT PEAKED':
                    not_peak_list.append(index)
                if peak_status == 'UNKNOWN':
                    unknown_peak_list.append(index)
        
        df['Use for dashboard?'].fillna('N', inplace = True)
        return df
    
    def read_in_cities_that_have_already_peaked():
        """
        Reads in dictionary of cities that have already peaked and their data sources
        INPUT: Link to text file with dictionary
        OUTPUT: Dictionry
        """
        with open('/Users/oliverwills/desktop/cities_to_have_peaked.txt','r', encoding = 'utf-8') as f:
            cities = simplejson.load(f)
        f.close()
        return cities

    def check_emissions_status_of_cities_that_have_peaked(cities, df):
        """
        Checks whether cities that have previously peaked are shown as having peaked. If not, and if PC4 is still
        True, the emissions status is set to PEAKED. 
        INPUT: Dictionary of cities and data sources
        OUTPUT: DataFrame
        """
        for city, data_source in cities.items():
            try:
                current_status = df[(df['City']==city) & (df['Use for dashboard?']=='Y')]['Peak Status'].values[0]
                current_index = df[(df['City']==city) & (df['Use for dashboard?']=='Y')]['Peak Status'].index[0]
                if current_status != 'PEAKED':
                    peak_index = df[(df['City']==city) & (df['Data Source'] == data_source)].index[0]
                    if df.loc[peak_index, 'PC4: Max emissions <10% higher than recent inventory']:
                        #peak_index = df[(df['City']==city) & (df['Data Source'] == data_source)].index[0]
                        #if df.loc[peak_index,'PC4: Max emissions <10% higher than recent inventory']:
                        df.loc[peak_index, 'Peak Status'] = 'PEAKED'
                        if peak_index != current_index:
                            df.loc[peak_index, 'Use for dashboard?'] = 'Y'
                            df.loc[current_index, 'Use for dashboard?'] = 'N'
            except:
                continue
        return df
                
    df = select_cities(df)
    cities_that_have_already_peaked = read_in_cities_that_have_already_peaked()
    df = check_emissions_status_of_cities_that_have_peaked(cities_that_have_already_peaked, df)
    update_cities_that_have_peaked(df,cities_that_have_already_peaked)
    return df

In [50]:
def update_cities_that_have_peaked(df, cities_that_have_already_peaked):
    """
    Updates text file with dictionary of cities and data sources that have peaked
    INPUT: DataFrame
    OUTPUT: Dictionary written to text file
    """
    df = df[(df['Use for dashboard?'] == 'Y')&(df['Peak Status']=='PEAKED')]
    
    cities = list(df['City'])
    cities_to_have_peaked = dict(zip(df['City'], df['Data Source']))
    
    with open('/Users/oliverwills/desktop/cities_to_have_peaked.txt', 'w', encoding ='utf-8') as f:
        simplejson.dump(cities_to_have_peaked, f)
    f.close()

In [51]:
def reshape_data_for_dashboard(df):
    """
    Reshapes data for use in the Qlik dashboatd
    INPUT: DataFrame (Years as rows)
    OUTPUT: DataFrame (Years as column)
    """
    df = df[df['Use for dashboard?'] == 'Y']
    cols = df.columns.difference(['Verified by city','Num data points', 'Max emissions', 'Recent emissions', 'Recent emissions year', 'PC1: At least 3 year of data available', 'PC2: Max emissions >5 years before recent inventory', 'PC3: Recent inventory <5 years old', 'PC4: Max emissions <10% higher than recent inventory', 'Use for dashboard?'])
    df = df[cols] 
    df = pd.melt(df, id_vars=["City", "Data Source", "Protocol","Peak Status", "Max emissions year"], var_name="Year", value_name="Emissions").sort_values(['City', 'Year']).reset_index(drop=True)
    df['Peak year'] = df.apply(lambda x : 1 if x['Max emissions year'] == x['Year'] and x['Peak Status'] == 'PEAKED' else 0, axis=1)
    df.drop('Max emissions year', axis = 1, inplace = True)
    return df

In [52]:
def generate_dataframes(path_1, path_2):
    """
    Generates DataFrames used in the programme by calling above functions
    INPUT: File paths to 2017 Peaking Analysis and GPC Tracker
    OUTPUT: Tuple of 6 DataFrames
    """
    df1 = read_in_data_from_master_emissions_tracker(path_1)
    #df2 = read_in_data_from_gpc_tracker(path_2)
    #df3 = join_data_from_peaking_analysis_and_gpc_tracker(df1, df2)
    #df4 = combine_gpc_and_non_gpc_data_sources(df3)
    #df5 = calculate_peak_emissions(df4,current_year)
    #df6 = select_cities_to_use_in_dashboard(df5)
    #df7 = reshape_data_for_dashboard(df6)
    df2 = combine_gpc_and_non_gpc_data_sources(df1)
    df3 = calculate_peak_emissions(df2,current_year)
    df4 = select_cities_to_use_in_dashboard(df3)
    df5 = reshape_data_for_dashboard(df4)

    return (df1, df2, df3, df4, df5) #CHANGE df1 to df2 and df3

In [53]:
def write_to_excel(results):
    """
    Writes dataframes to Excel
    INPUT: Tuple of 6 DataFrames
    OUTPUT: None
    """
    #Create a Pandas Excel writer using XlsxWriter as the engine
    writer = pd.ExcelWriter('/Users/oliverwills/Box/00_Live App Raw Data/Peaking Analysis/peaking_emissions_dashboard.xlsx', engine='xlsxwriter')
    #Write each dataframe to a different worksheet
    results[3].to_excel(writer, sheet_name='MASTER_Peak_Emissions', index = False)
    results[4].to_excel(writer, sheet_name='DASHBOARD_Peak_Emissions', index = False)
    #Close the Pandas Excel writer and output the Excel file
    writer.save()

In [54]:
def run_programme(path_1, path_2):
    results = generate_dataframes(path_1, path_2)
    write_to_excel(results)
    return results

In [55]:
path_1 = '/Users/oliverwills/Box/C40 (internal)/Regions and Cities (internal)/M&P/04_Analytics/00_Raw data/01_Emissions/Live tracker/GHG Master Tracker_20190508_COPY2.xlsx'
#path_1 = '/Users/oliverwills/Box/C40 (internal)/Regions and Cities (internal)/M&P/04_Analytics/03_Other analytics/01_Peaking analysis/00_GCAS/GHG Master Tracker_20190513.xlsx'
path_2 = '/Users/oliverwills/Box/C40 (internal)/Regions and Cities (internal)/M&P/04_Analytics/00_Raw data/01_Emissions/Live tracker/01_GPC Inventory Tracker.xlsx'

In [56]:
results = run_programme(path_1, path_2)

In [57]:
results[0]

,City,Data Source,Protocol,Verified by city,Year,Emissions
0,Accra,1,C40_GPC,No,2015,2321904
3,Addis Ababa,3,CDP_GPC,No,2012,5043860
4,Amman,6,CDP_Other,No,2014,7215407
6,Amman,1,C40_GPC,No,2014,7431422
7,Amsterdam,7,City_Other,Yes,1990,3011000
8,Amsterdam,5,Target_Other,No,1990,3417000
26,Amsterdam,7,City_Other,Yes,2008,4008518
27,Amsterdam,7,City_Other,Yes,2009,4045686
29,Amsterdam,7,City_Other,No,2010,4485670
30,Amsterdam,7,City_Other,Yes,2010,4026047


In [58]:
results[1]

,City,Data Source,Protocol,Verified by city,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,Num data points,Max emissions,Max emissions year,Recent emissions,Recent emissions year,PC1: At least 3 year of data available,PC2: Max emissions >5 years before recent inventory,PC3: Recent inventory <5 years old,PC4: Max emissions <10% higher than recent inventory,Peak Status
0,Accra,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2321904,0,0,1,2321904,2015,2321904,2015,False,False,True,False,UNKNOWN
1,Addis Ababa,3,CDP_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5043860,0,0,0,0,0,1,5043860,2012,5043860,2012,False,False,False,False,UNKNOWN
2,Amman,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7431422,0,0,0,1,7431422,2014,7431422,2014,False,False,True,False,UNKNOWN
3,Amman,6,CDP_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7215407,0,0,0,1,7215407,2014,7215407,2014,False,False,True,False,UNKNOWN
4,Amsterdam,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4804359,0,0,1,4804359,2015,4804359,2015,False,False,True,False,UNKNOWN
5,Amsterdam,5,Target_Other,No,3417000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3417000,1990,3417000,1990,False,False,False,False,UNKNOWN
6,Amsterdam,6,CDP_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4138000,0,0,1,4138000,2015,4138000,2015,False,False,True,False,UNKNOWN
7,Amsterdam,7,City_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4485670,4428768,4520689,4492935,4478294,0,0,0,5,4520689,2012,4478294,2014,True,False,True,False,NOT PEAKED
8,Amsterdam,7,City_Other,Yes,3011000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4008518,4045686,4026047,3979708,4072819,4044060,4015035,4136792,3948153,3932562,11,4136792,2015,3932562,2017,True,False,True,False,NOT PEAKED
298,Amsterdam,8,All non GPC considered,No,3417000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4008518,4045686,4485670,4428768,4520689,4492935,4478294,4138000,3948153,3932562,11,4520689,2012,3932562,2017,True,True,True,True,PEAKED


In [59]:
results[2]

,City,Data Source,Protocol,Verified by city,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,Num data points,Max emissions,Max emissions year,Recent emissions,Recent emissions year,PC1: At least 3 year of data available,PC2: Max emissions >5 years before recent inventory,PC3: Recent inventory <5 years old,PC4: Max emissions <10% higher than recent inventory,Peak Status
0,Accra,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2321904,0,0,1,2321904,2015,2321904,2015,False,False,True,False,UNKNOWN
1,Addis Ababa,3,CDP_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5043860,0,0,0,0,0,1,5043860,2012,5043860,2012,False,False,False,False,UNKNOWN
2,Amman,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7431422,0,0,0,1,7431422,2014,7431422,2014,False,False,True,False,UNKNOWN
3,Amman,6,CDP_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7215407,0,0,0,1,7215407,2014,7215407,2014,False,False,True,False,UNKNOWN
4,Amsterdam,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4804359,0,0,1,4804359,2015,4804359,2015,False,False,True,False,UNKNOWN
5,Amsterdam,5,Target_Other,No,3417000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3417000,1990,3417000,1990,False,False,False,False,UNKNOWN
6,Amsterdam,6,CDP_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4138000,0,0,1,4138000,2015,4138000,2015,False,False,True,False,UNKNOWN
7,Amsterdam,7,City_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4485670,4428768,4520689,4492935,4478294,0,0,0,5,4520689,2012,4478294,2014,True,False,True,False,NOT PEAKED
8,Amsterdam,7,City_Other,Yes,3011000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4008518,4045686,4026047,3979708,4072819,4044060,4015035,4136792,3948153,3932562,11,4136792,2015,3932562,2017,True,False,True,False,NOT PEAKED
298,Amsterdam,8,All non GPC considered,No,3417000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4008518,4045686,4485670,4428768,4520689,4492935,4478294,4138000,3948153,3932562,11,4520689,2012,3932562,2017,True,True,True,True,PEAKED


In [60]:
results[3]

,City,Data Source,Protocol,Verified by city,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,Num data points,Max emissions,Max emissions year,Recent emissions,Recent emissions year,PC1: At least 3 year of data available,PC2: Max emissions >5 years before recent inventory,PC3: Recent inventory <5 years old,PC4: Max emissions <10% higher than recent inventory,Peak Status,Use for dashboard?
0,Accra,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2321904,0,0,1,2321904,2015,2321904,2015,False,False,True,False,UNKNOWN,Y
1,Addis Ababa,3,CDP_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5043860,0,0,0,0,0,1,5043860,2012,5043860,2012,False,False,False,False,UNKNOWN,Y
2,Amman,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7431422,0,0,0,1,7431422,2014,7431422,2014,False,False,True,False,UNKNOWN,Y
3,Amman,6,CDP_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7215407,0,0,0,1,7215407,2014,7215407,2014,False,False,True,False,UNKNOWN,N
4,Amsterdam,7,City_Other,Yes,3011000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4008518,4045686,4026047,3979708,4072819,4044060,4015035,4136792,3948153,3932562,11,4136792,2015,3932562,2017,True,False,True,False,NOT PEAKED,Y
5,Amsterdam,1,C40_GPC,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4804359,0,0,1,4804359,2015,4804359,2015,False,False,True,False,UNKNOWN,N
6,Amsterdam,5,Target_Other,No,3417000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3417000,1990,3417000,1990,False,False,False,False,UNKNOWN,N
7,Amsterdam,6,CDP_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4138000,0,0,1,4138000,2015,4138000,2015,False,False,True,False,UNKNOWN,N
8,Amsterdam,7,City_Other,No,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4485670,4428768,4520689,4492935,4478294,0,0,0,5,4520689,2012,4478294,2014,True,False,True,False,NOT PEAKED,N
9,Amsterdam,8,All non GPC considered,No,3417000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4008518,4045686,4485670,4428768,4520689,4492935,4478294,4138000,3948153,3932562,11,4520689,2012,3932562,2017,True,True,True,True,PEAKED,N


In [61]:
results[4]

,City,Data Source,Protocol,Peak Status,Year,Emissions,Peak year
0,Accra,1,C40_GPC,UNKNOWN,1990,0,0
1,Accra,1,C40_GPC,UNKNOWN,1991,0,0
2,Accra,1,C40_GPC,UNKNOWN,1992,0,0
3,Accra,1,C40_GPC,UNKNOWN,1993,0,0
4,Accra,1,C40_GPC,UNKNOWN,1994,0,0
5,Accra,1,C40_GPC,UNKNOWN,1995,0,0
6,Accra,1,C40_GPC,UNKNOWN,1996,0,0
7,Accra,1,C40_GPC,UNKNOWN,1997,0,0
8,Accra,1,C40_GPC,UNKNOWN,1998,0,0
9,Accra,1,C40_GPC,UNKNOWN,1999,0,0


In [62]:
results[4]['Peak year'].value_counts()

0    1848
1      28
Name: Peak year, dtype: int64

In [63]:
results[4][results[4]['Peak year'] == 1]

,City,Data Source,Protocol,Peak Status,Year,Emissions,Peak year
242,Barcelona,6,CDP_Other,PEAKED,2008,4053766,1
253,Berlin,7,City_Other,PEAKED,1991,29546000,1
323,Boston,2,City_GPC,PEAKED,2005,7370738,1
491,Chicago,2,City_GPC,PEAKED,2005,36700027,1
533,Copenhagen,7,City_Other,PEAKED,1991,3874137,1
644,Heidelberg,8,All non GPC considered,PEAKED,1990,915000,1
934,London,7,City_Other,PEAKED,2000,50313969,1
952,Los Angeles,2,City_GPC,PEAKED,1990,36200000,1
997,Madrid,7,City_Other,PEAKED,2007,16123000,1
1028,Melbourne,2,City_GPC,PEAKED,2010,5713905,1
